In [425]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [426]:
filepath = '.\input\BA_2006-01-01_to_2018-01-01.csv' # CHANGE THE FILE PATH FOR WHICH YOU WANT TO START PREDICTING
data = pd.read_csv(filepath)
data = data.sort_values('Date')
# data.head(10)

In [427]:

import matplotlib.pyplot as plt
import seaborn as sns
"""
sns.set_style("darkgrid")
plt.figure(figsize = (15,9))
plt.plot(data[['Close']])
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.title("YOUR Stock Price",fontsize=18, fontweight='bold')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price (USD)',fontsize=18)
plt.show()
"""


'\nsns.set_style("darkgrid")\nplt.figure(figsize = (15,9))\nplt.plot(data[[\'Close\']])\nplt.xticks(range(0,data.shape[0],500),data[\'Date\'].loc[::500],rotation=45)\nplt.title("YOUR Stock Price",fontsize=18, fontweight=\'bold\')\nplt.xlabel(\'Date\',fontsize=18)\nplt.ylabel(\'Close Price (USD)\',fontsize=18)\nplt.show()\n'

In [428]:

price = data[['Close']]
price.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3020 entries, 0 to 3019
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   3020 non-null   float64
dtypes: float64(1)
memory usage: 47.2 KB


In [429]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

C:\Users\Frank\AppData\Local\Temp\ipykernel_19320\68737012.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [430]:
def split_data(stock, lookback): # will need to be modified to fit project
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data)
    test_set_size = int(np.round(0.2*data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    # return [x_train, y_train, x_test, y_test]
    return [x_train, y_train, x_test, y_test]

In [431]:
lookback = 20 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)
# print('x_train.shape = ',x_train.shape)
# print('y_train.shape = ',y_train.shape)
# print('x_test.shape = ',x_test.shape)
# print('y_test.shape = ',y_test.shape)

In [432]:
import torch
import torch.nn as nn

x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [433]:

input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100


In [434]:

import matplotlib.pyplot as plt
import seaborn as sns
"""
sns.set_style("darkgrid")
plt.figure(figsize = (15,9))
plt.plot(data[['Close']])
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.title("Amazon Stock Price",fontsize=18, fontweight='bold')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price (USD)',fontsize=18)
plt.show()
"""


'\nsns.set_style("darkgrid")\nplt.figure(figsize = (15,9))\nplt.plot(data[[\'Close\']])\nplt.xticks(range(0,data.shape[0],500),data[\'Date\'].loc[::500],rotation=45)\nplt.title("Amazon Stock Price",fontsize=18, fontweight=\'bold\')\nplt.xlabel(\'Date\',fontsize=18)\nplt.ylabel(\'Close Price (USD)\',fontsize=18)\nplt.show()\n'

In [435]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out


In [436]:
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [437]:
#FUUNY IMPORT SPACE
import time

In [438]:

hist = np.zeros(num_epochs)
start_time = time.time()
gru = []


model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

# THE MODEL HAS BEEN LOADED
model = torch.load('sussy.model')
model.eval()

y_train_pred = model(x_train)
predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_gru.detach().numpy()))


#loss = criterion(y_train_pred, y_train_gru)
# loss.backward()



In [439]:

predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_gru.detach().numpy()))

In [440]:
import seaborn as sns
"""

sns.set_style("darkgrid")    

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot(1, 2, 1)
ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (GRU)", color='tomato')
ax.set_title('Stock price', size = 14, fontweight='bold')
ax.set_xlabel("Days", size = 14)
ax.set_ylabel("Cost (USD)", size = 14)
ax.set_xticklabels('', size=10)
"""

'\n\nsns.set_style("darkgrid")    \n\nfig = plt.figure()\nfig.subplots_adjust(hspace=0.2, wspace=0.2)\n\nplt.subplot(1, 2, 1)\nax = sns.lineplot(x = original.index, y = original[0], label="Data", color=\'royalblue\')\nax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (GRU)", color=\'tomato\')\nax.set_title(\'Stock price\', size = 14, fontweight=\'bold\')\nax.set_xlabel("Days", size = 14)\nax.set_ylabel("Cost (USD)", size = 14)\nax.set_xticklabels(\'\', size=10)\n'

In [441]:
import math, time
from sklearn.metrics import mean_squared_error

# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train_gru.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test_gru.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
gru.append(trainScore)
gru.append(testScore)

Train Score: 2.44 RMSE
Test Score: 5.42 RMSE


In [442]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(price)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred

# shift test predictions for plotting
testPredictPlot = np.empty_like(price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred

original = scaler.inverse_transform(price['Close'].values.reshape(-1,1))

#predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = testPredictPlot # the one of the two lines must be uncommented

predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions)

In [443]:
# import plotly.express as px
# import plotly.graph_objects as go

# predictions = np.append(trainPredictPlot, testPredictPlot, axis=1) must be uncommented for this to work
"""
fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='FUTURE Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (GRU)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show()
"""


"\nfig = go.Figure()\nfig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],\n                    mode='lines',\n                    name='Train prediction')))\nfig.add_trace(go.Scatter(x=result.index, y=result[1],\n                    mode='lines',\n                    name='FUTURE Test prediction'))\nfig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],\n                    mode='lines',\n                    name='Actual Value')))\nfig.update_layout(\n    xaxis=dict(\n        showline=True,\n        showgrid=True,\n        showticklabels=False,\n        linecolor='white',\n        linewidth=2\n    ),\n    yaxis=dict(\n        title_text='Close (USD)',\n        titlefont=dict(\n            family='Rockwell',\n            size=12,\n            color='white',\n        ),\n        showline=True,\n        showgrid=True,\n        showticklabels=True,\n        linecolor='white',\n        linewidth=2,\n        ticks='outside',\n        tickfont=dict(\n            family=

In [444]:
gru = pd.DataFrame(gru, columns=['GRU'])
result

,0,1
0,NaN,70.44
1,NaN,71.17
2,NaN,70.33
3,NaN,69.35
4,NaN,68.77
...,...,...
3015,290.754883,295.10
3016,290.926727,295.36
3017,291.040100,295.62
3018,291.158173,296.35


In [445]:
result[1] #THIS IS WHAT YOU WANT. REMOVE THE FIRST THOUSAND OR SO LINES

0        70.44
1        71.17
2        70.33
3        69.35
4        68.77
         ...  
3015    295.10
3016    295.36
3017    295.62
3018    296.35
3019    294.91
Name: 1, Length: 3020, dtype: float64